In [1]:
from chi import server, context, lease
import os

context.version = "1.0" 
key_name = "pub_key_wl3187"
context.choose_project()
context.choose_site(default="CHI@UC") # change this according to lease server

In [2]:
l = lease.get_lease(f"gpu_rtx6000_project_3_grp6") # change this lease name
l.show()

HTML(value='\n        <h2>Lease Details</h2>\n        <table>\n            <tr><th>Name</th><td>gpu_rtx6000_pr…

Lease Details:
Name: gpu_rtx6000_project_3_grp6
ID: dd06053c-fd8a-48de-b700-89bd2ffe3209
Status: ACTIVE
Start Date: 2025-04-23 16:00:00
End Date: 2025-04-24 01:00:00
User ID: d06284a26f7aab2b313235db5087bbcd8d435120201888829b2377849270e4db
Project ID: 7c0a7a1952e44c94aa75cae1ff5dc9b4

Node Reservations:
ID: d5792a46-0be9-483b-8f5b-0613df5ae3f8, Status: active, Min: 1, Max: 1

Floating IP Reservations:

Network Reservations:

Events:


In [3]:
username = os.getenv('USER') # all exp resources will have this prefix
s = server.Server(
    f"node-project-2-group6",  # change it
    reservation_id=l.node_reservations[0]["id"],
    image_name="CC-Ubuntu24.04-CUDA",
    key_name=key_name # change to my public key name here
)
s.submit(idempotent=True)

Waiting for server node-project-2-group6's status to become ACTIVE. This typically takes 10 minutes, but can take up to 20 minutes.


Server has moved to status ACTIVE


Attribute,node-project-2-group6
Id,6059dc29-f69d-42c7-baf4-ace2a3ccab39
Status,ACTIVE
Image Name,CC-Ubuntu24.04-CUDA
Flavor Name,baremetal
Addresses,sharednet1: IP: 10.140.81.75 (v4) Type: fixed MAC: 24:6e:96:7e:0b:28
Network Name,sharednet1
Created At,2025-04-23T16:03:08Z
Keypair,pub_key_wl3187
Reservation Id,d5792a46-0be9-483b-8f5b-0613df5ae3f8
Host Id,c15c5d0cd98629a41c320d11364f137f4320899eed52f609fb88500c


In [4]:
s.associate_floating_ip()

In [5]:
s.refresh()
s.check_connectivity()

Checking connectivity to 192.5.87.126 port 22.


Connection successful


s.refresh()
s.show(type="widget")

Retrieve code from repo

In [6]:
s.refresh()
s.show(type="widget")

Attribute,node-project-2-group6
Id,6059dc29-f69d-42c7-baf4-ace2a3ccab39
Status,ACTIVE
Image Name,CC-Ubuntu24.04-CUDA
Flavor Name,baremetal
Addresses,sharednet1: IP: 10.140.81.75 (v4) Type: fixed MAC: 24:6e:96:7e:0b:28 IP: 192.5.87.126 (v4) Type: floating MAC: 24:6e:96:7e:0b:28
Network Name,sharednet1
Created At,2025-04-23T16:03:08Z
Keypair,pub_key_wl3187
Reservation Id,d5792a46-0be9-483b-8f5b-0613df5ae3f8
Host Id,c15c5d0cd98629a41c320d11364f137f4320899eed52f609fb88500c


In [7]:
# security_groups = [
#   {'name': "allow-ssh", 'port': 22, 'description': "Enable SSH traffic on TCP port 22"},
#   {'name': "allow-5000", 'port': 5000, 'description': "Enable TCP port 5000 (used by Flask)"},
#   {'name': "allow-8000", 'port': 8000, 'description': "Enable TCP port 8000 (used by FastAPI)"},
#   {'name': "allow-8888", 'port': 8888, 'description': "Enable TCP port 8888 (used by Jupyter)"},
#   {'name': "allow-3000", 'port': 3000, 'description': "Enable TCP port 3000 (used by Grafana)"},
#   {'name': "allow-9090", 'port': 9090, 'description': "Enable TCP port 9090 (used by Prometheus)"},
#   {'name': "allow-9000", 'port': 9000, 'description': "Enable TCP port 9000 (used by minio 1)"},
#     {'name': "allow-9001", 'port': 9001, 'description': "Enable TCP port 9001 (used by minio 2)"},
#     {'name': "allow-6379", 'port': 6379, 'description': "Enable TCP port 6379 (used by ray head 1)"},
#      {'name': "allow-8265", 'port': 8265, 'description': "Enable TCP port 8265 (used by ray head 2)"},
# ]

In [8]:
from chi import server, context
import chi, os, time, datetime

# configure openstacksdk for actions unsupported by python-chi
os_conn = chi.clients.connection()
nova_server = chi.nova().servers.get(s.id)

# Get current security groups
current_sg_names = [group.name for group in nova_server.list_security_group()]
print(f"Current security groups: {current_sg_names}")

# Define the ports you need
needed_ports = [22, 5000, 8000, 8888, 3000, 9090, 9000, 9001, 6379, 8265]

# Get existing security group rules to avoid duplicates
existing_ports = []
for sg_name in current_sg_names:
    try:
        sg = os_conn.get_security_group(sg_name)
        if sg:
            # Handle the case where security_group_rules might be a list of dicts
            rules = sg.get('security_group_rules', [])
            for rule in rules:
                if isinstance(rule, dict):
                    # If it's a dictionary
                    if (rule.get('direction') == 'ingress' and 
                        rule.get('protocol') == 'tcp' and 
                        rule.get('port_range_min') is not None):
                        existing_ports.append(rule.get('port_range_min'))
                else:
                    # If it has attributes
                    try:
                        if (getattr(rule, 'direction', '') == 'ingress' and 
                            getattr(rule, 'protocol', '') == 'tcp' and 
                            getattr(rule, 'port_range_min', None) is not None):
                            existing_ports.append(rule.port_range_min)
                    except Exception as e:
                        print(f"Error accessing rule attributes: {e}")
    except Exception as e:
        print(f"Error processing security group {sg_name}: {e}")

print(f"Ports already covered: {existing_ports}")

# Find ports that still need to be added
ports_to_add = [port for port in needed_ports if port not in existing_ports]
print(f"Ports still needing rules: {ports_to_add}")

# Only create a new security group if there are ports to add
if ports_to_add:
    # Create a descriptive name based on the ports being added
    sg_name = f"additional-ports"
    
    if not os_conn.get_security_group(sg_name):
        os_conn.create_security_group(sg_name, "Additional required ports")
        
        # Add rules for each needed port
        for port in ports_to_add:
            os_conn.create_security_group_rule(
                sg_name, 
                port_range_min=port, 
                port_range_max=port, 
                protocol='tcp', 
                remote_ip_prefix='0.0.0.0/0'
            )
            print(f"Added rule for port {port}")
    
        # Add the security group to the server
        nova_server.add_security_group(sg_name)
        print(f"Added security group: {sg_name}")
    
print(f"Final security groups: {[group.name for group in nova_server.list_security_group()]}")

Current security groups: ['default']
Ports already covered: [22]
Ports still needing rules: [5000, 8000, 8888, 3000, 9090, 9000, 9001, 6379, 8265]
Final security groups: ['default']


In [9]:
s.execute("git clone --branch dev_infra https://github.com/eus-lwq/leximind.git")

/opt/conda/lib/python3.10/site-packages/paramiko/client.py:889: UserWarning: Unknown ssh-ed25519 host key for 192.5.87.126: b'3dccaff01fbedf437d852e095bd0fc44'
  warnings.warn(
Cloning into 'leximind'...


<Result cmd='git clone --branch dev_infra https://github.com/eus-lwq/leximind.git' exited=0>

In [10]:
# setup docker
s.execute("curl -sSL https://get.docker.com/ | sudo sh")
s.execute("sudo groupadd -f docker; sudo usermod -aG docker $USER")

# Executing docker install script, commit: 53a22f61c0628e58e1d6680b49e82993d304b449


+ sh -c apt-get -qq update >/dev/null
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install ca-certificates curl >/dev/null

Running kernel seems to be up-to-date.

The processor microcode seems to be up-to-date.

Restarting services...
 systemctl restart packagekit.service

No containers need to be restarted.

No user sessions are running outdated binaries.

No VM guests are running outdated hypervisor (qemu) binaries on this host.
+ sh -c install -m 0755 -d /etc/apt/keyrings
+ sh -c curl -fsSL "https://download.docker.com/linux/ubuntu/gpg" -o /etc/apt/keyrings/docker.asc
+ sh -c chmod a+r /etc/apt/keyrings/docker.asc
+ sh -c echo "deb [arch=amd64 signed-by=/etc/apt/keyrings/docker.asc] https://download.docker.com/linux/ubuntu noble stable" > /etc/apt/sources.list.d/docker.list
+ sh -c apt-get -qq update >/dev/null
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install docker-ce docker-ce-cli containerd.io docker-compose-plugin docker-ce-rootless-extras docker-buildx-pl

Client: Docker Engine - Community
 Version:           28.1.1
 API version:       1.49
 Go version:        go1.23.8
 Git commit:        4eba377
 Built:             Fri Apr 18 09:52:14 2025
 OS/Arch:           linux/amd64
 Context:           default

Server: Docker Engine - Community
 Engine:
  Version:          28.1.1
  API version:      1.49 (minimum version 1.24)
  Go version:       go1.23.8
  Git commit:       01f442b
  Built:            Fri Apr 18 09:52:14 2025
  OS/Arch:          linux/amd64
  Experimental:     false
 containerd:
  Version:          1.7.27
  GitCommit:        05044ec0a9a75232cad458027ca83437aae3f4da
 runc:
  Version:          1.2.5
  GitCommit:        v1.2.5-0-g59923ef
 docker-init:
  Version:          0.19.0
  GitCommit:        de40ad0


To run Docker as a non-privileged user, consider setting up the
Docker daemon in rootless mode for your user:

    dockerd-rootless-setuptool.sh install

Visit https://docs.docker.com/go/rootless/ to learn about rootless mode.


T

<Result cmd='sudo groupadd -f docker; sudo usermod -aG docker $USER' exited=0>

In [11]:
# setup nvidia-container
s.execute("curl -fsSL https://nvidia.github.io/libnvidia-container/gpgkey | sudo gpg --dearmor -o /usr/share/keyrings/nvidia-container-toolkit-keyring.gpg \
  && curl -s -L https://nvidia.github.io/libnvidia-container/stable/deb/nvidia-container-toolkit.list | \
    sed 's#deb https://#deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://#g' | \
    sudo tee /etc/apt/sources.list.d/nvidia-container-toolkit.list")
s.execute("sudo apt update")
s.execute("sudo apt-get install -y nvidia-container-toolkit")
s.execute("sudo nvidia-ctk runtime configure --runtime=docker")
# for https://github.com/NVIDIA/nvidia-container-toolkit/issues/48
s.execute("sudo jq 'if has(\"exec-opts\") then . else . + {\"exec-opts\": [\"native.cgroupdriver=cgroupfs\"]} end' /etc/docker/daemon.json | sudo tee /etc/docker/daemon.json.tmp > /dev/null && sudo mv /etc/docker/daemon.json.tmp /etc/docker/daemon.json")
s.execute("sudo systemctl restart docker")
s.execute("sudo apt-get update")
s.execute("sudo apt-get install docker-compose-plugin")

deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://nvidia.github.io/libnvidia-container/stable/deb/$(ARCH) /
#deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://nvidia.github.io/libnvidia-container/experimental/deb/$(ARCH) /


Hit:1 https://download.docker.com/linux/ubuntu noble InRelease
Get:2 https://nvidia.github.io/libnvidia-container/stable/deb/amd64  InRelease [1477 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2404/x86_64  InRelease
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu noble InRelease [256 kB]
Hit:5 http://security.ubuntu.com/ubuntu noble-security InRelease
Get:6 https://nvidia.github.io/libnvidia-container/stable/deb/amd64  Packages [17.7 kB]
Hit:7 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates InRelease
Get:8 http://nova.clouds.archive.ubuntu.com/ubuntu noble-backports InRelease [126 kB]
Fetched 401 kB in 1s (318 kB/s)
Reading package lists...
Building dependency tree...
Reading state information...
282 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libnvidia-container-tools libnvidia-container

debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Fetched 5848 kB in 2s (3473 kB/s)
Selecting previously unselected package libnvidia-container1:amd64.
(Reading database ... 110605 files and directories currently installed.)
Preparing to unpack .../libnvidia-container1_1.17.5-1_amd64.deb ...
Unpacking libnvidia-container1:amd64 (1.17.5-1) ...
Selecting previously unselected package libnvidia-container-tools.
Preparing to unpack .../libnvidia-container-tools_1.17.5-1_amd64.deb ...
Unpacking libnvidia-container-tools (1.17.5-1) ...
Selecting previously unselected package nvidia-container-toolkit-base.
Preparing to unpack .../nvidia-container-toolkit-base_1.17.5-1_amd64.deb ...
Unpacking nvidia-container-toolkit-base (1.17.5-1) ...
Selecting previously unselected package nvidia-container-toolkit.
Preparing to unpack .../nvidia-container-toolkit_1.17.5-1_amd64.deb ...
Unpacking nvidia-container-toolkit (1.17.5-1) ...
Setting up nvidia-container-toolkit-base (1.17.5-1) ...
Setting up libnvidia-container1:amd64 (1.17.5-1) ...
Setting up lib

debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

Running kernel seems to be up-to-date.

The processor microcode seems to be up-to-date.

No services need to be restarted.

No containers need to be restarted.

No user sessions are running outdated binaries.

No VM guests are running outdated hypervisor (qemu) binaries on this host.
time="2025-04-23T16:14:52Z" level=info msg="Config file does not exist; using empty config"
time="2025-04-23T16:14:52Z" level=info msg="Wrote updated config to /etc/docker/daemon.json"
time="2025-04-23T16:14:52Z" level=info msg="It is recommended that docker daemon be restarted."


Hit:1 https://download.docker.com/linux/ubuntu noble InRelease
Get:2 http://nova.clouds.archive.ubuntu.com/ubuntu noble InRelease [256 kB]
Hit:3 http://security.ubuntu.com/ubuntu noble-security InRelease
Hit:4 https://nvidia.github.io/libnvidia-container/stable/deb/amd64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2404/x86_64  InRelease
Get:6 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates InRelease [126 kB]
Hit:7 http://nova.clouds.archive.ubuntu.com/ubuntu noble-backports InRelease
Fetched 382 kB in 1s (304 kB/s)
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
docker-compose-plugin is already the newest version (2.35.1-1~ubuntu.24.04~noble).
0 upgraded, 0 newly installed, 0 to remove and 282 not upgraded.


<Result cmd='sudo apt-get install docker-compose-plugin' exited=0>

In [12]:
# now connect with terminal: ssh -i ~/.ssh/id_rsa cc@A.B.C.D